# Multi-tenant Chat with Papers - Query papers
## Get keys and urls

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print(WEAVIATE_URL[:10])
print(WEAVIATE_KEY[:10])
print(OPENAI_API_KEY[:10])

cwzujlbgtk
L1FUbzRPaU
sk-proj-iu


## Connect to Weaviate

In [2]:
import weaviate
from weaviate.classes.init import Auth
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),

    headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY
    },

    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=2, query=45, insert=120),  # Values in seconds
    # )
)

client.is_ready()

True

## Vector search on tenants

In [3]:
papers = client.collections.get("Papers")

ten = papers.with_tenant("2212-10496")

response = ten.query.near_text(
    query="Unsupervised learning",
    limit=5,
)

for item in response.objects:
    print(item.properties["chunk"])

many follow-up research generally consider the Transfer Learning setup where the dense re- triever is ﬁrst learned using a diverse and richly supervised corpus and query collection, namely MS-MARCO (Thakur et al., 2021; Wang et al., 2022; Yu et al., 2022). However, as stated by Izacard et al. (2021), such a large collection can rarely be assumed. In this paper, therefore, we study the problem of building effective dense retrieval systems without relevance labels. Similar to Izacard et al. (2021), we also do not assume access to the test time corpora for training. This is a more realistic setup and prevents over-engineering on the test corpora. By the deﬁnition in Sachan et al. (2022), our setup can be roughly considered as “unsuper- vised” . Strictly,
supervision is not available, we start by examining self-supervised representa- tion learning methods. Modern deep learning en- ables two distinct learning algorithms. At the token level, generative large language models (LLM) pre- traine

## Generative Search with tenants

In [4]:
papers = client.collections.get("Papers")

ten2212 = papers.with_tenant("2212-10496")

response = ten2212.generate.near_text(
    query="Unsupervised learning",
    limit=5,
    single_prompt="What does the following text describe: {chunk}",
)

for item in response.objects:
    print(item.properties["chunk"])
    print(item.generative.text, '\n')

many follow-up research generally consider the Transfer Learning setup where the dense re- triever is ﬁrst learned using a diverse and richly supervised corpus and query collection, namely MS-MARCO (Thakur et al., 2021; Wang et al., 2022; Yu et al., 2022). However, as stated by Izacard et al. (2021), such a large collection can rarely be assumed. In this paper, therefore, we study the problem of building effective dense retrieval systems without relevance labels. Similar to Izacard et al. (2021), we also do not assume access to the test time corpora for training. This is a more realistic setup and prevents over-engineering on the test corpora. By the deﬁnition in Sachan et al. (2022), our setup can be roughly considered as “unsuper- vised” . Strictly,
The text describes a research study focusing on the development of effective dense retrieval systems in the context of Transfer Learning. It emphasizes the challenge of building such systems without relying on relevance labels or a large,

In [5]:
papers = client.collections.get("Papers")

ten2212 = papers.with_tenant("2212-10496")

response = ten2212.generate.near_text(
    query="Unsupervised learning",
    limit=5,
    grouped_task="Explain how unsupervised learning works. Use only the provided content.",
    grouped_properties=["chunk"]
)

for item in response.objects:
    print(item.properties["chunk"])

print(response.generative.text)

many follow-up research generally consider the Transfer Learning setup where the dense re- triever is ﬁrst learned using a diverse and richly supervised corpus and query collection, namely MS-MARCO (Thakur et al., 2021; Wang et al., 2022; Yu et al., 2022). However, as stated by Izacard et al. (2021), such a large collection can rarely be assumed. In this paper, therefore, we study the problem of building effective dense retrieval systems without relevance labels. Similar to Izacard et al. (2021), we also do not assume access to the test time corpora for training. This is a more realistic setup and prevents over-engineering on the test corpora. By the deﬁnition in Sachan et al. (2022), our setup can be roughly considered as “unsuper- vised” . Strictly,
supervision is not available, we start by examining self-supervised representa- tion learning methods. Modern deep learning en- ables two distinct learning algorithms. At the token level, generative large language models (LLM) pre- traine

In [6]:
def paper_rag(paper_id, query, prompt):
    papers = client.collections.get("Papers")
    ten = papers.with_tenant(paper_id)

    response = ten.generate.near_text(
        query=query,
        limit=5,
        grouped_task=prompt + " Use only the provided content.",
        grouped_properties=["chunk"],
    )

    return {
        "title": response.objects[0].properties["title"],
        "source": [p.properties["chunk"] for p in response.objects],
        "generated": response.generative.text
    }

In [7]:
paper_rag(
    "2212-10496",
    "Unsupervised learning",
    "Explain how unsupervised learning works"
)

{'title': 'Precise Zero-Shot Dense Retrieval without Relevance Labels',
 'source': ['many follow-up research generally consider the Transfer Learning setup where the dense re- triever is ﬁrst learned using a diverse and richly supervised corpus and query collection, namely MS-MARCO (Thakur et al., 2021; Wang et al., 2022; Yu et al., 2022). However, as stated by Izacard et al. (2021), such a large collection can rarely be assumed. In this paper, therefore, we study the problem of building effective dense retrieval systems without relevance labels. Similar to Izacard et al. (2021), we also do not assume access to the test time corpora for training. This is a more realistic setup and prevents over-engineering on the test corpora. By the deﬁnition in Sachan et al. (2022), our setup can be roughly considered as “unsuper- vised” . Strictly,',
  'supervision is not available, we start by examining self-supervised representa- tion learning methods. Modern deep learning en- ables two distinct l

In [8]:
papers = client.collections.get("Papers")
papers.tenants.get()

{'2212-10496': TenantOutput(name='2212-10496', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 '2401-00107': TenantOutput(name='2401-00107', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>)}

## Close the client

In [9]:
client.close()